# Python для анализа данных

## Библиотеки для работы с данными в табличном формате: pandas. SQL для Python. Работа с Clickhouse. 

Автор: *Ян Пиле, НИУ ВШЭ*

Мы с вами уже немного поработали с запросами данных из интернета, причем как непосредственно с сайтов, так и через некоторые API. Давайте теперь попробуем поработать с SQL прямо из Python.

### Порешаем задачи

In [ ]:
import json # Чтобы разбирать поля
import requests # чтобы отправлять запрос к базе
import pandas as pd # чтобы в табличном виде хранить результаты запроса

Нужно написать функцию, которая будет отправлять текст SQL-запроса, в нашем случае - запроса к ClickHouse, на сервер, а по выполнении запроса забирать его результаты в каком-то виде.

In [ ]:
# имена явки пароли. если хотите, чтобы считалось с вашего логина, вставьте сюда свои логин и пароль
USER = 'student'
PASS = 'dpo_python_2020'
HOST = 'http://hse.beslan.pro:8080/'

def get_clickhouse_data(query,
                        host=HOST, 
                        USER = USER, 
                        PASS = PASS, 
                        connection_timeout = 1500, 
                        dictify=True, 
                        **kwargs):
    NUMBER_OF_TRIES = 5  # Количество попыток запуска
    DELAY = 10           #время ожидания между запусками
    import time
    params = kwargs      #если вдруг нам нужно в функцию положить какие-то параметры
    if dictify:
        query += "\n FORMAT JSONEachRow"   # dictify = True отдает каждую строку в виде JSON'a

    for i in range(NUMBER_OF_TRIES):

        headers = {'Accept-Encoding': 'gzip'}

        r = requests.post(host, 
                          params = params, 
                          auth=(USER, PASS), 
                          timeout = connection_timeout, 
                          data=query
                          )              # отправили запрос на сервер

        if r.status_code == 200 and not dictify:    
            return r.iter_lines()         # генератор :)
        elif r.status_code == 200 and dictify:
            return (json.loads(x) for x in r.iter_lines()) # генератор :)
        
        else:
            print('ATTENTION: try #%d failed' % i)
            if i != (NUMBER_OF_TRIES - 1):
                print(r.text)
                time.sleep(DELAY * (i + 1))
            else:
                raise(ValueError, r.text)

Функция ниже преобразует полученные нами данные из генератора в pd.Dataframe

In [4]:
query = """
select  *
from default.events
limit 10
"""

In [ ]:
d = get_clickhouse_data(query, dictify=True)

In [ ]:
next(d)

In [3]:
def get_data(query):
    return pd.DataFrame(list(get_clickhouse_data(query, dictify=True)))

In [ ]:
get_data(query)

Предлагаю немного разобраться в структуре нашей базы. Давайте достанем по 5-10 строк каждой из таблиц и посмотри, что же в них лежит. В events, например, уложены AppPlatform - Платформа (операционная система мобильного устройства), events - количество событий, произошедших в эту дату (будем, например, считать, что события это клики и каждый из них платный), EventDate - Дата события, DeviceID - идентификатор устройства.

In [10]:
# впуливаем сюда запрос
query = """
select  *
from default.events
limit 10
"""

In [11]:
f = get_data(query)

In [12]:
f

,AppPlatform,events,EventDate,DeviceID
0,android,8,2019-09-29,7429291373250434008
1,android,175,2019-09-15,7429291824672902510
2,android,0,2019-09-17,7429291824672902510
3,android,0,2019-09-26,7429291824672902510
4,android,4,2019-04-29,7429292273953361459
5,android,38,2019-08-20,7429293114537639018
6,android,38,2019-05-21,7429298825563999474
7,android,4,2019-05-26,7429298825563999474
8,android,100,2019-08-07,7429300397574411770
9,android,26,2019-01-31,7429301272237917347


Только что мы научились превращать результат нашего SQL-запроса в PANDAS-dataframe.

В devices, например, уложены UserID - идентификатор пользователя, DeviceID - идентификатор устройства.

In [ ]:
query = """
SELECT *
from devices
limit 10
"""
get_data(query)

Проверим, однозначное ли это соответствие (может ли у человека быть два устройства и могут ли с одного устройства сидеть два человека)

In [ ]:
query = """
SELECT UserID, uniqExact(DeviceID) as cnt
from devices
group by UserID
having cnt>1
"""
get_data(query)

In [ ]:
query = """
SELECT DeviceID, uniqExact(UserID) as cnt
from devices
group by DeviceID
having cnt>1
"""
get_data(query)

Видим, что оба запроса возвращают пустой результат. Это означает, что соответствие между UserID и DeviceID взаимно-однозначное. Это позволит нам избежать многих проблем впоследствии. 

В checks хранится стоимость всех покупок одного UserID за день, BuyDate - дата покупки, Rub - стоимость покупки

In [8]:
query = """
SELECT Rub, BuyDate, UserID
from checks
order by Rub desc
limit 10
"""
get_data(query)

,Rub,BuyDate,UserID
0,181063,2019-06-04,15605251414578189718
1,66318,2019-09-16,9613023063681218234
2,56327,2019-01-18,10398072986532593332
3,48758,2019-08-19,16967914029594691558
4,42945,2019-09-13,3216678490746804369
5,38989,2019-05-31,287271529430059152
6,38555,2019-04-03,6890024450198999904
7,38189,2019-03-22,17121049924398614106
8,35769,2019-02-05,9002023542637148786
9,34396,2019-02-26,5050480259433864828


Проверим, есть ли записи, у которых набору UserID-BuyDate соответствует несколько записей

In [ ]:
query = """
SELECT BuyDate, UserID, count(*) as cnt
from checks
group by BuyDate, UserID
having cnt>1
"""
get_data(query)

Нам снова повезло! На каждого человека и каждый день в таблицу пишется суммарная стоимость его покупок. Теперь посмотрим на таблицу installs. 

В ней InstallationDate - дата установки, InstallCost - стоимость установки, Platform - Платформа (операционная система мобильного устройства), DeviceID - идентификатор устройства и Source - источник трафика (откуда человек пришел устанавливать приложение: сам нашел в поисковике, из рекламы, перешел по реферальной ссылке и т.д.)

In [ ]:
query = """
SELECT *
from installs
limit 10
"""
get_data(query)

Давайте сформулируем несколько задач, которые мы на этих данных хотим решить.

### В течение какого срока установка, в среднем, окупается, в зависимости от:
* платформы 
* источника трафика 

Воспользуйтесь тем периодом данных, который сочтете обоснованным для формулировки вывода

Давайте посмотрим среднюю стоимость установки в зависимости от источника трафика. Будем считать, что GMV мы считаем в валюте Rub\*5, а стоимость одного события равна 0.5 у.е.

Для начала достанем информацию, которая касается установок приложения и приклеим к ней информацию о том, какой UserID какие установки совершил. Для этого надо сделать join таблиц installs и devices. Я предлагаю считать данные за 1 квартал 2019 (почему бы и нет). В отображении остаывим 10 записей, чтобы экран не заполнять лишним. Join сделаем inner, предполагая, что нет таких DeviceID, которые никому не принадлежат (хотя вообще говоря, это стоит проверить)

In [ ]:
query = """
select a.Source as Source, 
    a.InstallationDate as InstallationDate, 
    a.InstallCost as InstallCost, 
    a.DeviceID as DeviceID,
    b.UserID as UserID
from installs as a
inner join devices as b
on a.DeviceID = b.DeviceID
where InstallationDate between '2019-01-01' and '2019-03-31'
limit 10
"""

res = get_data(query)
res

Теперь нам нужно посчитать суммарную стоимость всех заказов, которые указанный UserID сделал за этот квартал (это как раз один из двух источников нашей доходной части), а расходную часть - InstallCost- мы уже достали. Здесь необходимо делать left join, потому что могут быть люди, которые ничего не купили за этот период, хоть и установили приложение. Значит наше условие ограничения на left join должно брать только те покупки людей, которые произошли от даты установки до конца квартала, а также оставлять записи, в которых не было ни одной покупки, это можно обеспечить условием BuyDate is null (в правой таблице не нашлось ни одной покупки). После того, как мы эту информацию приджойнили, посчитаем на каждый факт установки суммарную стоимость всех покупок с помощью функции sum(). Мы также хотим, чтоб при суммировании у тех, кто не купил ничего в поле GMV - Gross Merchandise Value (суммарный оборот заказов)- стоял ноль. Для этого мы сначала переведем содержимое поля Rub в интересующую нас валюту (мы договорились умножать его на 5), а потом суммировать не само получившееся значение, а coalesce(Rub\*5,0) эта функция возвращает первое непустое значение из списка своих аргументов. Получается, что если поле Rub заполнено, она вернет Rub\*5, а если человек ничего не купил, то она вернет 0, как раз этого мы и добивались. Стоит заметить, что в качестве левой таблицы для join'а мы вставили наш предыдущий запрос.

In [ ]:
query = """
select a.Source as Source, 
        a.InstallationDate as InstallationDate, 
        a.InstallCost as InstallCost, 
        a.DeviceID as DeviceID,
        b.UserID as UserID,
        sum(coalesce(b.Rub*5, 0)) as GMV
from  (select a.Source as Source, 
        a.InstallationDate as InstallationDate, 
        a.InstallCost as InstallCost, 
        a.DeviceID as DeviceID,
        b.UserID as UserID
    from installs as a
    inner join devices as b
    on a.DeviceID = b.DeviceID
    where InstallationDate between '2019-01-01' and '2019-03-31') as a
left join checks as b
    on a.UserID = b.UserID
where (b.BuyDate >= a.InstallationDate
    and b.BuyDate<='2019-03-31')
    or b.BuyDate is null
group by a.Source , 
        a.InstallationDate, 
        a.InstallCost, 
        a.DeviceID,
        b.UserID
limit 10
"""

res = get_data(query)
res

Остается предпоследний шаг: таким же образом собрать информацию о произошедших событиях (они лежат в поле events таблицы events и мы договорились, что стоимость одного события - 0.5 у.е.). Полностью повторим логику, которая у нас была до этого. Только в этот раз попробуем в функцию sum() не подставлять coalesce. Если мы уверены, что в каждом Source произошло хотя бы одно событие, то в итоговом результате наша сумма будет точно ненулевой. 

In [ ]:
query = """
select a.Source as Source, 
            a.InstallationDate as InstallationDate, 
            a.InstallCost as InstallCost, 
            a.DeviceID as DeviceID,
            a.UserID as UserID,
            a.GMV as GMV,
            sum(events*0.5) as events_revenue
from   (select a.Source as Source, 
            a.InstallationDate as InstallationDate, 
            a.InstallCost as InstallCost, 
            a.DeviceID as DeviceID,
            b.UserID as UserID,
            sum(coalesce(b.Rub*5, 0)) as GMV
    from  (select a.Source as Source, 
            a.InstallationDate as InstallationDate, 
            a.InstallCost as InstallCost, 
            a.DeviceID as DeviceID,
            b.UserID as UserID
        from installs as a
        inner join devices as b
        on a.DeviceID = b.DeviceID
        where InstallationDate between '2019-01-01' and '2019-03-31') as a
    left join checks as b
        on a.UserID = b.UserID
    where (b.BuyDate >= a.InstallationDate
        and b.BuyDate<='2019-03-31')
        or b.BuyDate is null
    group by a.Source , 
            a.InstallationDate, 
            a.InstallCost, 
            a.DeviceID,
            b.UserID) as a
left join events as b
on a.DeviceID = b.DeviceID
where (b.EventDate >= a.InstallationDate
        and b.EventDate<='2019-03-31')
        or b.EventDate is null
group by a.Source as Source, 
            a.InstallationDate as InstallationDate, 
            a.InstallCost as InstallCost, 
            a.DeviceID as DeviceID,
            a.UserID as UserID,
            a.GMV as GMV
limit 10
"""

res = get_data(query)
res

Ну и теперь произведем финальный шаг: суммируем все по источникам трафика и сразу посчитаем ROI - суммарный доход/суммарные затраты

In [ ]:
query = """

select Source, uniqExact(UserID) as users,
    SUM(InstallCost) AS InstallCost,
    sum(GMV) as GMV,
    SUM(events_revenue) AS events_revenue
from  (select a.Source as Source, 
                a.InstallationDate as InstallationDate, 
                a.InstallCost as InstallCost, 
                a.DeviceID as DeviceID,
                a.UserID as UserID,
                a.GMV as GMV,
                sum(events*0.5) as events_revenue
    from   (select a.Source as Source, 
                a.InstallationDate as InstallationDate, 
                a.InstallCost as InstallCost, 
                a.DeviceID as DeviceID,
                b.UserID as UserID,
                sum(coalesce(b.Rub*5, 0)) as GMV
        from  (select a.Source as Source, 
                a.InstallationDate as InstallationDate, 
                a.InstallCost as InstallCost, 
                a.DeviceID as DeviceID,
                b.UserID as UserID
            from installs as a
            inner join devices as b
            on a.DeviceID = b.DeviceID
            where InstallationDate between '2019-01-01' and '2019-03-31') as a
        left join checks as b
            on a.UserID = b.UserID
        where (b.BuyDate >= a.InstallationDate
            and b.BuyDate<='2019-03-31')
            or b.BuyDate is null
        group by a.Source , 
                a.InstallationDate, 
                a.InstallCost, 
                a.DeviceID,
                b.UserID) as a
    left join events as b
    on a.DeviceID = b.DeviceID
    where (b.EventDate >= a.InstallationDate
            and b.EventDate<='2019-03-31')
            or b.EventDate is null
    group by a.Source as Source, 
                a.InstallationDate as InstallationDate, 
                a.InstallCost as InstallCost, 
                a.DeviceID as DeviceID,
                a.UserID as UserID,
                a.GMV as GMV
    )
group by Source
"""
res = get_data(query)
res

С помощью pandas приведем поля к нужному нам формату (По умолчанию Clickhouse выплевывает результаты в строковом формате)

In [ ]:
res = res.astype({'users':int, 'InstallCost':float, 'GMV':float, 'events_revenue':float})

Также посчитаем доходную часть

In [ ]:
res['Profit'] = res['GMV'] + res['events_revenue']

И, наконец, посчитаем ROI

In [ ]:
res['ROI'] = res['Profit']/res['InstallCost']
res

Что мы видим на примере данных:
    
    1. Бесплатные каналы привлечения приносят большую часть наш доходов
    2. Среди платных каналов не окупилось всего два, да и то крайне малые.
    3. Крупные платные каналы Source9 и Source14 имеют сильно отличающийся ROI - можно поразбираться, почему так
    
Аналогичное распределение можно построить по платформам (iOS/Android), предлагаю сделать это самостоятельно.
Также на этих данных можно построить так называемую RFM-сегментацию пользователей, прочитать можно тут:

https://www.owox.ru/blog/use-cases/rfm-analysis/

https://en.wikipedia.org/wiki/RFM_(market_research)
    
    